# Code source

copy/pasted from https://github.com/SaulMontoya/interactiveShapefileExplorationonJupyterlab

In [ ]:
import os, json
from pathlib import Path
import geopandas as gpd
from ipywidgets import widgets, Layout
from ipyleaflet import Map, GeoData, basemaps, LayersControl, ScaleControl, FullScreenControl, WidgetControl, basemap_to_tiles, DivIcon
from ipywidgets import widgets, IntSlider, jslink

In [ ]:
#get shapefile name list

shapes = Path('smoke_extents/')
foo = shapes.glob('**/*.shp')

floo = {f.stem: str(f) for f in foo}
#shpList = [x[:-4] for x in os.listdir('../shp') if x[-3:]=='shp']


In [ ]:
map_layout = Layout(width='750px', height='750px')
geodata_style = {'color': 'black', 'weight': 1, 'fillColor': '#' + '3' * 6}

In [ ]:

#dropdown widget
selDrop = widgets.Dropdown(
    options=sorted(floo.keys()),
    value=sorted(floo.keys())[0],
    description='Shapefile:',
    disabled=False,
)

display(selDrop)

#button widget
selBot = widgets.Button(
    description='Select shapefile',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Select shapefile',
    icon='check'
)

display(selBot)

#map object
m = Map(center=(40,-96), zoom = 4,
        layout=map_layout,
        basemap = basemaps.NASAGIBS.ViirsEarthAtNight2012) #basemaps.OpenStreetMap.Mapnik) #OpenTopoMap)

zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=4)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widgetControl = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widgetControl)

m.add_layer(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik, opacity=1))
#m.add_layer(basemaps.NASAGIBS.ViirsEarthAtNight2012)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl(position='topright'))
m.add_control(LayersControl())
display(m)

#show map function
def showMap(selShp):
    global geoData, geoDf
    selDf = gpd.read_file(floo[selShp])


    geoDf = selDf.to_crs(4326)
#     lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean()/2
#     latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean()/2

#     m.center = (latCent,lonCent)
    geoData = GeoData(geo_dataframe=geoDf, name=selShp, style=geodata_style)
    m.add_layer(geoData)
    
#on click function
def on_button_clicked(b):
    showMap(selDrop.value)

selBot.on_click(on_button_clicked)

In [ ]:
from time import sleep
import numpy as np

for i in np.linspace(0.2, 0.01, num=10):
    the_style = dict(geoData.style)
    the_style["fillOpacity"] = i
    the_style["opacity"] = i
    #geoData.style['opacity'] = 1 / (1 + i)
    geoData.style = the_style
    sleep(1)

In [ ]:
def cross_fade(layer1, layer2, fade_duration=1, max_opacity=0.2, steps=30):
    for opacity in np.linspace(max_opacity, 0, num=steps):
        style1 = dict(layer1.style)
        style2 = dict(layer2.style)
        for key in ["fillOpacity", "opacity"]:
            style1[key] = opacity
            style2[key] = max_opacity - opacity
        layer1.style = style1
        layer2.style = style2
        sleep(fade_duration / steps)
    

In [ ]:
date_label = DivIcon(html='<h1 style="color:red">MOOOO</h1>', icon_size=(50, 50))
date_label2 = DivIcon(html="<h1>HFDIUHFIUSDHFUIS</h1>", icon_size=(50, 50))


In [ ]:
date_marker = Marker(location=(46.873889, -96.767222), icon=date_label)
m.add(date_marker);

In [ ]:
date_marker.icon = date_label2

In [ ]:
date_label.html = "<h1>HFDIUHFIUSDHFUIS</h1>"

In [ ]:
selDf = gpd.read_file(floo["hms_smoke20230506"])

zero_opacity = {"fillOpacity": 0, "opacity": 0}
zero_opacity.update(geodata_style)

geoDf2 = selDf.to_crs(4326)
#     lonCent = (geoDf.bounds.maxx + geoDf.bounds.minx).mean()/2
#     latCent = (geoDf.bounds.maxy + geoDf.bounds.miny).mean()/2

#     m.center = (latCent,lonCent)
geoData2 = GeoData(geo_dataframe=geoDf2, name="hms_smoke20230506", style=zero_opacity)
m.add(geoData2);

In [ ]:
def animate(shape_file_dict, date_box):
    names = sorted(shape_file_dict.keys())
    this_df = gpd.read_file(shape_file_dict[names[0]])
    this_geo_data = GeoData(geo_dataframe=this_df, name=names[0], style=geodata_style)
    m.add(this_geo_data)
    for that in names[1:]:
        that_df = gpd.read_file(shape_file_dict[that])
        that_geo_data = GeoData(geo_dataframe=that_df, name=that, style=zero_opacity)
        m.add(that_geo_data)
        cross_fade(this_geo_data, that_geo_data, fade_duration=0.2)
        m.remove(this_geo_data)
        this_geo_data = that_geo_data
        

In [ ]:
icon_bison = Icon(icon_url="https://thenounproject.com/api/private/icons/5844572/edit/?backgroundShape=SQUARE&backgroundShapeColor=%23000000&backgroundShapeOpacity=0&exportSize=752&flipX=false&flipY=false&foregroundColor=%23000000&foregroundOpacity=1&imageFormat=png&rotation=0",
                  icon_size=(100, 100))
icon_bison_brown = Icon(icon_url="https://thenounproject.com/api/private/icons/5844572/edit/?backgroundShape=SQUARE&backgroundShapeColor=%23000000&backgroundShapeOpacity=0&exportSize=752&flipX=false&flipY=false&foregroundColor=%238B4513&foregroundOpacity=1&imageFormat=png&rotation=0",
                        icon_size=(60, 60))
icon_obs = Icon(icon_url="https://thenounproject.com/api/private/icons/1336/edit/?backgroundShape=SQUARE&backgroundShapeColor=%23000000&backgroundShapeOpacity=0&exportSize=752&flipX=false&flipY=false&foregroundColor=%23000000&foregroundOpacity=1&imageFormat=png&rotation=0")
icon_obs_yellow = Icon(icon_url="https://thenounproject.com/api/private/icons/1336/edit/?backgroundShape=SQUARE&backgroundShapeColor=%23000000&backgroundShapeOpacity=0&exportSize=752&flipX=false&flipY=false&foregroundColor=%23FFd65B&foregroundOpacity=1&imageFormat=png&rotation=0")
moorhead = Marker(location=(46.873889, -96.767222), icon=icon_obs_yellow, title="Moorhead", alt="Moorhead", name="Moorhead")
m.add(moorhead)
trnp = Marker(location=(46.912222, -103.525278), icon=icon_bison_brown, name="TRNP")
m.add(trnp);

In [ ]:
animate(floo)

In [ ]:
cross_fade(geoData, geoData2, fade_duration=0.1)

In [ ]:
da_layer.name = "Moorhead"

In [ ]:
[g for g in dir(geoData) if not g.startswith("_")]

In [ ]:
geoData.style

In [ ]:
geoData.style = {'color': 'black', 'weight': 1, 'fillColor': '#' + '3' * 6}

In [ ]:
from ipyleaflet import Marker, Icon

In [ ]:
set(geoDf["Density"])

In [ ]:
from datetime import date, timedelta

In [ ]:
d = date(2023,5,1)
td = timedelta(days=30)
d, d + td

In [ ]:
date.today()